# Binding Prediction via API

http://tools.iedb.org/main/tools-api/

## MHC I

In [37]:
import requests
import csv

In [61]:
# HLA allele reference set (27 alleles long)
hla_ref_set_mhc1_27 = 'HLA-A*01:01,HLA-A*02:01,HLA-A*02:03,HLA-A*02:06,HLA-A*03:01,HLA-A*11:01,HLA-A*23:01,HLA-A*24:02,HLA-A*26:01,HLA-A*30:01,HLA-A*30:02,HLA-A*31:01,HLA-A*32:01,HLA-A*33:01,HLA-A*68:01,HLA-A*68:02,HLA-B*07:02,HLA-B*08:01,HLA-B*15:01,HLA-B*35:01,HLA-B*40:01,HLA-B*44:02,HLA-B*44:03,HLA-B*51:01,HLA-B*53:01,HLA-B*57:01,HLA-B*58:01'

# length must be set for each allele separately
length_mch1_27 = '9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9'


# HLA allele reference set (27 alleles long * 2, one for each length 9 and 10)
hla_ref_set_mhc1_54 = hla_ref_set_mhc1_27 + "," + hla_ref_set_mhc1_27

# length must be set for each allele separately
length_mch1_54 = '9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10'




In [59]:
def read_data(filename):
    
    # read in single sequence
    with open(filename, mode='r') as read_file:
        return read_file.readline().strip('\n')
    
    
def predict_mhc1(a_set,l_set,sequence):
    '''
    Note that the default value for species is human.
    '''
    
    # define post
    data = {'method':'recommended',
            'allele':a_set,
            'length':l_set,
            'sequence_text':sequence}

    site = 'http://tools-cluster-interface.iedb.org/tools_api/mhci/'  

    # perform prediction
    return requests.post(site, data=data)


def write_csv(response, filename):
    
    with open(filename, mode='w') as write_file:
        file_writer = csv.writer(write_file, delimiter=',')

        for each_row in response.text.split('\n'):
            file_writer.writerow(each_row.split('\t'))

In [ ]:
# perform prediction for each sequence
s = read_data('data/Binding_Prediction/test_sequence2.txt')
r = predict_mhc1(hla_ref_set_mhc1_54,length_mch1_54,s)
write_csv(r, 'data/Binding_Prediction/test_result3.csv')

In [50]:
# combine results into single csv